In [3]:
import os
import openai

openai.organization = "org-40iTy1feGwMOIzcYyoUTX6mk"
openai.api_key = os.getenv("OPENAI_API_KEY")

## Read pdf file and classify sections

In [4]:
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        contents = []
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            contents.append(page.extract_text())
        return contents

# Specify the path to your PDF file
pdf_file_path = '/Users/krystalgong/Downloads/test.pdf'
pdf_file_path2 = '/Users/krystalgong/Downloads/test2.pdf'

# Read the PDF file and extract its contents
pdf_contents = read_pdf(pdf_file_path2)

# Print the extracted contents
text = ""
for page_num, content in enumerate(pdf_contents):
    print(f"Page {page_num + 1}: {content}")
    text += content

Page 1: Organizational Behavior and Human Decision Processes 167 (2021) 72–87
Available online 22 July 2021
0749-5978/© 2021 Elsevier Inc. All rights reserved.Using fresh starts to nudge increased retirement savings 
John Beshearsa,*, Hengchen Daib, Katherine L. Milkmanc, Shlomo Benartzid 
aNegotiation, Organizations & Markets, Harvard Business School, Boston, MA, USA 
bManagement and Organizations, UCLA Anderson School of Management, Los Angeles, CA, USA 
cOperations, Information and Decisions, University of Pennsylvania Wharton School, Philadelphia, PA, USA 
dBehavioral Decision Making, UCLA Anderson School of Management, Los Angeles, CA, USA   
ARTICLE INFO  
Keywords: 
Choice architecture 
Nudge 
Randomized field experiment 
Fresh start 
Savings ABSTRACT  
We conducted a field experiment to study the effect of framing future moments in time as new beginnings (or 
“fresh starts ”). University employees (N 6,082) received mailings with an opportunity to choose between 
increasing th

In [6]:
def classify_section(text):
    prompt = f"""Paragraph: {text}

            Question: Which sections of the academic paper does the above paragraph likely belong to? 
                Give me top two one-word answers and make the most likely one in the first place. The answer format: [section1], [section2]
            """
    
    response = openai.Completion.create(
    engine='text-davinci-003',  # Specify the GPT-3.5 model
    prompt=prompt,
    max_tokens=100,  # Adjust the value based on the desired response length
    temperature=0.5,  # Adjust the temperature to control the randomness of the response
    n=1,  # Generate a single completion
    stop=None,  # Stop generating after the completion is finished
)

    section = response.choices[0].text.strip()[8:].split(', ')
    return section


method_lst = []
result_lst = []
method_start, method_end, result_start, result_end = None, None, None, None
for i, content in enumerate(pdf_contents):
    section = classify_section(content)
    # method and result should be in one block of text we should only find method if the next section also contains method
    if ('Methods' in section or 'Methodology' in section) and method_start is None:
        method_start = i
    if ('Results' in section or 'Conclusion' in section) and method_start is not None:
        if result_start is None:
            method_end = i-1
            result_start = i
        else:
            result_end = i
    print(f"Page{i+1}: {section}")

print(method_start, method_end, result_start, result_end)

Page1: ['Abstract', 'Introduction']
Page2: ['Introduction', 'Findings']
Page3: ['Introduction', 'Results']
Page4: ['Introduction', 'Methodology']
Page5: ['Introduction', 'Methodology']
Page6: ['Introduction', 'Methodology']
Page7: ['Introduction', 'Methodology']
Page8: ['Introduction', 'Methodology']
Page9: ['Results', 'Methodology']
Page10: ['Introduction', 'Results']
Page11: ['Introduction', 'Results']
Page12: ['Methodology', 'Results']
Page13: ['Introduction', 'Methodology']
Page14: ['Results', 'Discussion']
Page15: ['Acknowledgements', 'References']
Page16: ['Introduction', 'Conclusion']
3 7 8 15


In [7]:
result_section = "".join(pdf_contents[result_start:result_end+1])

## Yes/No questions via sliding window

In [8]:
def sliding_window(content, question, window_size=8985):
    content_length = len(content)

    for i in range(0, content_length, window_size//2):
        chunk = content[i : i + window_size]
    
        if len(content) < window_size:
            chunk = content

        prompt = f"""
        Paragraph: {chunk}
        
        Question: {question}
        """

        # Generate a completion using the OpenAI API
        response = openai.Completion.create(
            engine='text-davinci-003',  # Specify the GPT-3.5 model
            prompt=prompt,
            max_tokens=100,  # Adjust the value based on the desired response length
            temperature=0.5,  # Adjust the temperature to control the randomness of the response
            n=1,  # Generate a single completion
            stop=None,  # Stop generating after the completion is finished
        )

        # Retrieve the generated completion
        answer = response.choices[0].text.strip()[8:]

        if len(content) < window_size:
            return answer
        
        # Yes/No questions
        if answer[:3] == "Yes":
            return answer 
    return 'No'

### answer questions

In [10]:
answers = {}

In [11]:
results_type_continuous = "Are results evaluated with a comparison of means (continuous DV)? This means reporting the average and standard deviation, by condition. \
In a statistical test, this will typically be compared to the null hypothesis that their difference = 0."

answers[results_type_continuous] = sliding_window(result_section, results_type_continuous)

In [12]:
results_type_2by2 = "Are results evaluated with a comparison of counts across conditions (binary DV)? This means reporting the number doing the behavior and not doing it, by condition.\
In a statistical test, this will typically be compared to the null hypothesis that their difference = 0."

answers[results_type_2by2] = sliding_window(result_section, results_type_2by2) # wrong answer

In [13]:
results_type_binary = "Are results evaluated with a comparison of proportions (binary DV)? This means reporting the percent doing the behavior, by condition.\
In a statistical test, this will typically be compared to the null hypothesis that their difference = 0."

answers[results_type_binary] = sliding_window(result_section, results_type_binary) # wrong answer

In [14]:
results_type_diffprepostcont = "Are results evaluated with a comparison of pre- and post- averages (continuous DV)? This means reporting the average and standard deviation before the intervention and after the intervention, by condition.\
In a statistical test, this will typically be compared to the null hypothesis that their difference = 0."

answers[results_type_diffprepostcont] = sliding_window(result_section, results_type_diffprepostcont)

In [15]:
# results_focal_comparator = """For this behavior, which condition is the main "comparison" condition? This is the condition against which other conditions are compared in statistical tests. It is often the control condition. 

# If there is no primary comparison condition that the authors designate, pick the one that is closest to a control condition in this setting, to which other conditions are MOST sensibly compared.

# Note that for the columns to the right, you will capture ONLY statistical tests comparing a given condition to this comparator. If you wrote "Y" below for a condition, it will be blank for those columns; if you wrote "N" below for a condition, document any statistical tests comparing it to the condition with a "Y"."""


results_focal_comparator = """Is there a primary "comparison" condition in the behavior? This condition would be the main one against which other conditions
 are compared in statistical tests, often serving as the control condition. 
 If the authors haven't designated a primary comparison condition, is there one that closely resembles a control condition in this context, 
 and to which other conditions are most logically compared? Answer with 'Yes' if such a condition exists and 'No' otherwise"""
 
answers[results_focal_comparator] = sliding_window(result_section, results_focal_comparator)

In [16]:
results_type_regression = 'Are results evaluated with a regression? (Typically compared to the null hypothesis that beta = 0.)'
answers[results_type_regression] = sliding_window(result_section, results_type_regression)

In [17]:
results_reg_diffindiff = 'If it is a regression, is it a difference in differences regression?'
answers[results_reg_diffindiff] = sliding_window(result_section, results_reg_diffindiff)